#Planning under uncertainty in temporal availability constraints: Electric Vehicles Context.

## Genetic Algorithm (this is the version used in experiments)

In [ ]:
import random

# Número de filas de datos que deseas generar
num_filas = 20

# Nombre del archivo de salida
nombre_archivo = "vehiculos.txt"

# Abre el archivo en modo escritura
with open(nombre_archivo, "w") as archivo:
    for i in range(1, num_filas + 1):
        # Genera valores aleatorios para cada campo
        campo1 = i
        campo2 = random.randint(100, 200)
        campo3 = random.randint(8, 14)
        campo4 = random.randint(9, 18)
        campo5 = 0

        # Escribe la línea en el archivo
        linea = f"{campo1}, {campo2}, {campo3}, {campo4}, {campo5}\n"
        archivo.write(linea)

print(f"Se ha generado el archivo '{nombre_archivo}' con datos aleatorios.")

In [ ]:
# Class representing an electric vehicle
class ElectricVehicle:
    def __init__(self, id, distance, available_time_start, available_time_end, current_charge, battery_capacity, charge_speed, discharge_rate):
        self.id = id  # Vehicle identifier
        self.distance = distance  # Distance the vehicle needs to travel
        self.available_time_start = available_time_start  # Start time available for charging
        self.available_time_end = available_time_end  # End time available for charging
        self.current_charge = current_charge  # Current charge level
        self.battery_capacity = battery_capacity  # Battery capacity
        self.charge_speed = charge_speed  # Charging speed
        self.discharge_rate = discharge_rate  # KW discharged per 100 km (efficiency)

    # Calculates and returns the time needed to charge the vehicle enough for the specified distance
    def get_charging_time(self):
        required_charge = self.distance * (self.discharge_rate/100)  # Calculation of the required charge for the distance to be covered

        # If the car does not have enough charge
        if self.current_charge < required_charge:
            required_charge -= self.current_charge  # Subtract the current charge
            return required_charge / self.charge_speed  # Time in hours required to charge the vehicle

        else:
            return 0

    # Calculates the time required to charge the vehicle
    def get_charging_cost(self, begin_time, end_time, hourly_prices):
        total_cost = 0
        current_time = begin_time
        while current_time < end_time:
            total_cost += self.charge_speed * hourly_prices[int(current_time)]
            current_time += 1
        return total_cost

    def __str__(self):
        return f"Vehicle {self.id}"




In [ ]:
# Class representing a element of solution list
class item_planlist:
    def __init__(self, vehiculo, begin_time, end_time, cargador ):
        self.vehiculo = vehiculo
        self.tiempo_inicio = begin_time
        self.tiempo_fin = end_time
        self.cargador = cargador


In [ ]:
# Class representing a charger
class Cargador:
    def __init__(self, id):
        self.id = id
        self.periodos_ocupado = []

    def set_estado(self, hora_inicio, hora_fin):
        self.periodos_ocupado.append((hora_inicio, hora_fin))

    def eliminar_estado(self, hora_inicio, hora_fin):
        self.periodos_ocupado = [(inicio, fin) for inicio, fin in self.periodos_ocupado if (inicio, fin) != (hora_inicio, hora_fin)]

    def liberar_cargador(self):
        self.periodos_ocupado = []

    def esta_ocupado(self, hora_inicio, hora_fin):
        for inicio, fin in self.periodos_ocupado:
            if inicio <= hora_fin and fin >= hora_inicio:
                return True
        return False

    def tiene_horas_ocupadas(self):
        return bool(self.periodos_ocupado)



In [ ]:
import random
import numpy as np

In [ ]:
#Calculates timespan
def calcular_timespan(solucion):
    tiempo_inicio_mas_temprano = min(item.tiempo_inicio for item in solucion)
    tiempo_fin_mas_tardio = max(item.tiempo_fin for item in solucion)
    return tiempo_fin_mas_tardio - tiempo_inicio_mas_temprano

In [ ]:
#Generates the initial population
def generar_poblacion_inicial(vehiculos, cargadores, population_size):

    #Set of solutions
    poblacion = []
    for _ in range(population_size):

        #Generate each solution
        solucion = []

        #Release chargers before generating a new solution.
        for i in range(len(cargadores)):
          cargadores[i].liberar_cargador()


        for vehiculo in vehiculos:
          #Check if the vehicle needs to be loaded
          if vehiculo.get_charging_time() > 0:

            tiempo_inicio = random.uniform(vehiculo.available_time_start, vehiculo.available_time_end-0.5)
            tiempo_fin = random.uniform(tiempo_inicio+0.5,tiempo_inicio+vehiculo.get_charging_time())
            tiempo_fin = min(tiempo_fin, vehiculo.available_time_end)  #Ensure that the loading time does not exceed the available time.

            cargador = None

            #Assign charger
            for i in range(len(cargadores)):
              if(cargadores[i].esta_ocupado(tiempo_inicio, tiempo_fin) == False):
                cargador = cargadores[i]
                cargadores[i].set_estado(tiempo_inicio, tiempo_fin)
                break

            if(cargador != None):
              it = item_planlist(vehiculo,tiempo_inicio,tiempo_fin,cargador)
              solucion.append(it)



        #dd solution to popilation
        poblacion.append(solucion)
    return poblacion


#Evaluate the population in terms of cost and timespan
def evaluar_poblacion(poblacion, hourly_prices, hora_llegada_media, hora_llegada_varianza, num_muestras=10):
    evaluaciones = []

    #For each solution in population
    for solucion in poblacion:
        coste_total = 0
        tiempo_total = 0
        timespan_ini = 0
        timespan_fin = 0
        acum_timespan_fin = 0
        acum_timespan_ini = 0

        #For each item (vehicle) in solution
        for i, item in enumerate(solucion):
            #Generate N random samples for the arrival time of each vehicle before the mean arrival time, hour_arrival_mean[i]
            muestras_llegada = np.random.normal(item.vehiculo.available_time_start, hora_llegada_varianza[i], num_muestras)
            coste_muestras = 0
            tiempo_muestras = 0


            #Generate mean arrival time following a probability distribution
            muestras_salida = np.random.normal(item.vehiculo.available_time_end, hora_llegada_varianza[i], num_muestras)

            #Calculate sum of elements
            suma = sum(muestras_salida)

            #Calculate number of elements
            cantidad_elementos = len(muestras_salida)

            #Calculate mean
            media_salida = suma / cantidad_elementos

            #Verify that no sample falls outside the vehicle's availability
            for muestra_llegada in muestras_llegada:
                tiempo_inicio = muestra_llegada
                tiempo_fin = min(muestra_llegada + item.vehiculo.get_charging_time(), media_salida)
                tiempo_carga = tiempo_fin - tiempo_inicio

                if tiempo_carga > 0:
                    coste_muestras += item.vehiculo.get_charging_cost(tiempo_inicio, tiempo_fin, hourly_prices)
                    tiempo_muestras += tiempo_carga

                if i == (len(solucion)-1):
                  acum_timespan_fin += tiempo_fin

                if i == 0:
                  acum_timespan_ini = tiempo_inicio

            coste_total += (coste_muestras / num_muestras)
            tiempo_total += tiempo_muestras / num_muestras
            timespan_fin = acum_timespan_fin / num_muestras
            timespan_ini = acum_timespan_ini / num_muestras

        timespan = timespan_fin - timespan_ini
        evaluaciones.append((solucion, coste_total, tiempo_total, timespan))

    return evaluaciones


#Select the best solutions for subsequent crossover
def seleccionar_padres(evaluaciones, num_padres):
    evaluaciones.sort(key=lambda x: (x[1], x[2]))
    padres = []
    for i in range(num_padres):
        padres.append(evaluaciones[i][0])

    return padres



#Crossover of parents
def cruzar_padres(padre1, padre2):
    hijo = []

    #Release the chargers before generating a new solution.
    for i in range(len(cargadores)):
        cargadores[i].liberar_cargador()

    for i in range(len(padre1)):

        tiempo_inicio_hijo = min(padre1[i].tiempo_inicio, padre2[i].tiempo_inicio)
        tiempo_fin_hijo = max(padre1[i].tiempo_fin, padre2[i].tiempo_fin)

        cargador = None
        #Assign charger
        for j in range(len(cargadores)):
            if(cargadores[j].esta_ocupado(tiempo_inicio_hijo, tiempo_fin_hijo) == False):
              cargador = cargadores[j]
              cargadores[j].set_estado(tiempo_inicio_hijo, tiempo_fin_hijo)
              break

        item = item_planlist(padre1[i].vehiculo, tiempo_inicio_hijo, tiempo_fin_hijo, cargador)

        if (cargador == None):
              item = item_planlist(padre1[i].vehiculo, padre1[i].tiempo_inicio, padre1[i].tiempo_fin, padre1[i].cargador)

        hijo.append(item)

    return hijo



#Introduce mutations
def mutar_hijo(hijo, mutation_rate):

    #Release chargers before generating a new solution.
    for i in range(len(cargadores)):
      cargadores[i].liberar_cargador()

    for i in range(len(hijo)):
        if random.uniform(0, 1) < mutation_rate:
            nuevo_tiempo_inicio = random.uniform(hijo[i].vehiculo.available_time_start, hijo[i].vehiculo.available_time_end - 0.5)
            nuevo_tiempo_fin = nuevo_tiempo_inicio + hijo[i].vehiculo.get_charging_time()
            nuevo_tiempo_fin = min(nuevo_tiempo_fin, hijo[i].vehiculo.available_time_end)

            if nuevo_tiempo_fin == nuevo_tiempo_inicio:
                nuevo_tiempo_fin = nuevo_tiempo_inicio + 0.5

            #Assign charger
            for j in range(len(cargadores)):
              if(cargadores[j].esta_ocupado(nuevo_tiempo_inicio, nuevo_tiempo_fin) == False):
                cargadores[j].set_estado(nuevo_tiempo_inicio, nuevo_tiempo_fin)
                hijo[i].tiempo_inicio = nuevo_tiempo_inicio
                hijo[i].tiempo_fin = nuevo_tiempo_fin
                hijo[i].cargador = cargadores[j]
                break

    return hijo


In [ ]:
import time

# Parameters of the genetic algorithm
population_size = 10
num_generations = 50
num_padres = 3
mutation_rate = 0.2

# Generate chargers list
cargadores = [Cargador(1), Cargador(2), Cargador(3), Cargador(4)]

# Example of vehicles and prices per hour - Arrival and departure time with mean and variance
# Example of vehicles
'''vehiculo1 = ElectricVehicle(1, 100, 9, 10, 2, 80, 10, 5)
vehiculo2 = ElectricVehicle(2, 200, 8, 9, 3, 100, 8, 4)
vehiculo3 = ElectricVehicle(3, 150, 12, 13, 1, 120, 12, 6)
vehiculo4 = ElectricVehicle(4, 200, 12, 13, 1, 120, 12, 6)
vehiculo5 = ElectricVehicle(5, 100, 14, 17, 2, 80, 10, 5)
vehiculo6 = ElectricVehicle(6, 200, 13, 18, 3, 100, 8, 4)
vehiculo7 = ElectricVehicle(7, 150, 12, 20, 1, 120, 12, 6)
vehiculo8 = ElectricVehicle(8, 200, 15, 22, 1, 120, 12, 6)
vehiculo9 = ElectricVehicle(9, 100, 9, 10, 2, 80, 10, 5)
vehiculo10 = ElectricVehicle(10, 200, 8, 20, 3, 100, 8, 4)
vehiculo11 = ElectricVehicle(11, 150, 12, 18, 1, 120, 12, 6)
vehiculo12 = ElectricVehicle(12, 200, 14, 20, 1, 120, 12, 6)
vehiculo13 = ElectricVehicle(13, 100, 21, 22, 2, 80, 10, 5)
vehiculo14 = ElectricVehicle(14, 200, 22, 23, 3, 100, 8, 4)
vehiculo15 = ElectricVehicle(15, 150, 20, 23, 1, 120, 12, 6)
vehiculo16 = ElectricVehicle(16, 200, 15, 22, 1, 120, 12, 6)
vehiculo17 = ElectricVehicle(17, 100, 9, 20, 2, 80, 10, 5)
vehiculo18 = ElectricVehicle(18, 200, 6, 10, 3, 100, 8, 4)
vehiculo19 = ElectricVehicle(19, 150, 4, 9, 1, 120, 12, 6)
vehiculo20 = ElectricVehicle(20, 200, 17, 23, 1, 120, 12, 6)

vehiculos = [vehiculo1, vehiculo2, vehiculo3, vehiculo4, vehiculo5, vehiculo6, vehiculo7, vehiculo8, vehiculo9, vehiculo10]'''


#Create vehicle objects from file
def create_vehicles_from_file(file_name):
    vehicles = []
    with open(file_name, 'r') as file:
        lines = file.readlines()
        for line in lines:
            data = line.strip().split(', ')
            if len(data) == 8:
                id, distance, available_time_start, available_time_end, current_charge, battery_capacity, charge_speed, discharge_rate = map(float, data)
                vehicle = ElectricVehicle(id, distance, available_time_start, available_time_end, current_charge, battery_capacity, charge_speed, discharge_rate)
                vehicles.append(vehicle)
    return vehicles

file_name = 'vehiculos.txt'
vehiculos = create_vehicles_from_file(file_name)

# Example of vehicles and prices per hour - Arrival and departure time with mean and variance
hora_llegada_media = [9, 8, 12, 12, 14, 13, 12, 15, 9, 8]
hora_llegada_varianza = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

#, 12, 14, 21, 22, 20, 15, 9, 6, 4, 17    , 1, 1, 1, 1, 1, 1, 1, 1, 1, 1   2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2 , 2, 2, 2    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

hourly_prices = [0.1, 0.2, 0.15, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.1, 0.2, 0.3, 0.35, 0.3, 0.25, 0.2, 0.15]

start_time = time.time()
# Generate initial population
poblacion = generar_poblacion_inicial(vehiculos, cargadores, population_size)

#Print population - Uncomment if necessary
'''print("POBLACION")
for sol in poblacion:
  for it in sol:
    print("identificador vehiculo: "+ str(it.vehiculo.id))
    print("inicio "+str(it.tiempo_inicio))
    print("fin "+str(it.tiempo_fin))
    print("cargador "+str(it.cargador.id))
    print("")

  print("-------------------------------------------------")'''

'''evaluaciones=evaluar_poblacion(poblacion,hourly_prices)
padres = seleccionar_padres(evaluaciones, num_padres)
padre1 = random.choice(padres)
padre2 = random.choice(padres)
hijo = cruzar_padres(padre1, padre2)
hijo = mutar_hijo(hijo, mutation_rate)'''



# Run genetic algorithm
for generation in range(num_generations):
    evaluaciones = evaluar_poblacion(poblacion, hourly_prices, hora_llegada_media, hora_llegada_varianza)
    padres = seleccionar_padres(evaluaciones, num_padres)

    nueva_poblacion = []
    for i in range(population_size):
        padre1 = random.choice(padres)
        padre2 = random.choice(padres)
        hijo = cruzar_padres(padre1, padre2)
        hijo_mutado = mutar_hijo(hijo, mutation_rate)
        nueva_poblacion.append(hijo_mutado)

    poblacion = nueva_poblacion

# Get the best solution found
evaluaciones = evaluar_poblacion(poblacion, hourly_prices, hora_llegada_media, hora_llegada_varianza)
evaluaciones.sort(key=lambda x: (x[1], x[2]))
mejor_sol = evaluaciones[0]
end_time = time.time()

print("#########################################################################")

# Print the solution
print("Coste total: " + str(mejor_sol[1]))
print("Tiempo total: " + str(mejor_sol[2]))
print("Timespan " + str(mejor_sol[3]) )
print("Planificación:")

for item in mejor_sol[0]:
  print(item.vehiculo.id)
  print(item.tiempo_inicio)
  print(item.tiempo_fin)
  print(item.cargador.id)
  print("")

execution_time = end_time - start_time
print(f"Tiempo de ejecución: {execution_time * 1000:.2f} milisegundos")



# STATISTICS TO EVALUATE SOLUTION QUALITY - Uncomment to execute

'''file_name = 'vehiculosprueba.txt'
vehiculos2 = create_vehicles_from_file(file_name)

i=0
cont=0
vehiculos_noc=0
tiempo_real_carga=0
coste_real_total = 0
coste_real=0
coste_penalizacion=0
timespan_penalizacion=0
for item in mejor_sol[0]:

  if item.vehiculo.id == vehiculos2[i].id:

    if item.tiempo_inicio<vehiculos2[i].available_time_start:
      cont=cont+1
      tiempo_real_carga = item.tiempo_fin - vehiculos2[i].available_time_start
      tiempo_real_ini= vehiculos2[i].available_time_start

      if vehiculos[i].get_charging_time()>tiempo_real_carga:
          vehiculos_noc=vehiculos_noc+1
          tiempo_penalizacion=vehiculos[i].get_charging_time()-tiempo_real_carga
          #Para los vehiculos que por llegar tarde no se carguen lo suficiente, se añade una penalización sumando el coste de cargar lo que les falta en el momento más caro del día
          #Tiempo que queda x velocidad de carga x precio de hora más cara
          coste_penalizacion=coste_penalizacion+(tiempo_penalizacion*vehiculos[i].charge_speed)*0.35

    else:
      tiempo_real_carga=item.tiempo_fin-item.tiempo_inicio
      tiempo_real_ini=item.tiempo_inicio

    coste_real=vehiculos[i].get_charging_cost(item.tiempo_inicio, item.tiempo_fin, hourly_prices)


    timespan_inicio=0
    timespan_fin=0

    if i==0:
      timespan_inicio=tiempo_real_ini

    else: timespan_fin=item.tiempo_fin+tiempo_penalizacion

  i=i+1
  coste_real_total = coste_real_total + coste_real/2 ++coste_penalizacion
  timespan=timespan_fin-timespan_inicio

print("Número de intentos de carga y que no esté el vehículo")
print(cont)

print("Número de vehículos que no se cargan lo suficiente")
print(vehiculos_noc)

print("Coste real total")
print(coste_real_total)

print("Timespan real")
print(timespan)'''











-----------------------------------------------------------------

## Linear programing algorithm
Introducing a priori uncertainty

In [ ]:
import random
import numpy as np
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, value

class Vehicle:
    def __init__(self, distance, battery_capacity, current_charge, charging_speed, efficiency):
        self.distance = distance
        self.battery_capacity = battery_capacity
        self.current_charge = current_charge
        self.charging_speed = charging_speed
        self.efficiency = efficiency

class Charger:
    def __init__(self, charger_id, available_hours):
        self.charger_id = charger_id
        self.available_hours = available_hours
        self.vehicles_assigned = []

def occupied_hours(charger, hour):
    for vehicle in charger.vehicles_assigned:
        if vehicle[0] <= hour < vehicle[1]:
            return True
    return False

def optimize_charging_schedule_stochastic(vehicles, chargers, hourly_prices, num_samples):
    total_cost_sum = 0
    total_time_sum = 0

    for _ in range(num_samples):
        charged_vars = {}

        problem = LpProblem("Charging_Schedule_Optimization", LpMinimize)

        for vehicle in vehicles:
            for charger in chargers:
                for hour in range(charger.available_hours):
                    var_name = f"Vehicle_{vehicle.distance}_Charger_{charger.charger_id}_Hour_{hour}"
                    charged_vars[(vehicle, charger, hour)] = LpVariable(var_name, lowBound=0, upBound=1, cat="Integer")

        # Constraint: Each vehicle must be charged enough to cover the distance
        for vehicle in vehicles:
            required_energy = vehicle.distance / (vehicle.efficiency/100)
            problem += lpSum(charged_vars[(vehicle, charger, hour)] * vehicle.charging_speed for charger in chargers for hour in range(charger.available_hours)) >= required_energy

        # Constraint: Charging time must be within available time for each vehicle and charger
        for vehicle in vehicles:
            for charger in chargers:
                problem += lpSum(charged_vars[(vehicle, charger, hour)] for hour in range(charger.available_hours)) <= vehicle.battery_capacity / vehicle.charging_speed

        # Constraint: Each charger can charge one vehicle at a time
        for charger in chargers:
            for hour in range(charger.available_hours):
                problem += lpSum(charged_vars[(vehicle, charger, hour)] for vehicle in vehicles) <= 1

        # Constraint: Each vehicle can be charged by at most one charger at a time
        for vehicle in vehicles:
            for hour in range(charger.available_hours):
                problem += lpSum(charged_vars[(vehicle, charger, hour)] for charger in chargers) <= 1

        # Constraint: Charged energy must be sufficient to cover the distance for each vehicle
        for vehicle in vehicles:
            for charger in chargers:
                problem += lpSum(charged_vars[(vehicle, charger, hour)] * vehicle.charging_speed for hour in range(charger.available_hours)) >= vehicle.distance / (vehicle.efficiency/100)

        # Constraint: Total charging time must be within available time for each vehicle
        for vehicle in vehicles:
            for charger in chargers:
                problem += lpSum(charged_vars[(vehicle, charger, hour)] for hour in range(charger.available_hours)) * vehicle.charging_speed <= vehicle.battery_capacity

        # Objective function: minimize total cost and total time
        total_cost = LpVariable("Total_Cost", lowBound=0)
        total_time = LpVariable("Total_Time", lowBound=0)
        problem += total_cost + total_time

        # Constraint: calculate total cost based on charging schedules and hourly prices
        for vehicle in vehicles:
            for hour in range(charger.available_hours):
                problem += lpSum(charged_vars[(vehicle, charger, hour)] * hourly_prices[hour] for charger in chargers) == total_cost

        # Constraint: calculate total time based on charging schedules
        for vehicle in vehicles:
            for hour in range(charger.available_hours):
                problem += lpSum(charged_vars[(vehicle, charger, hour)] for charger in chargers) * vehicle.charging_speed == total_time

        # Solve the problem
        problem.solve()

        # Extract the charging schedules and total cost
        charging_schedule = []
        for vehicle in vehicles:
            for charger in chargers:
                schedule = [(hour, charged_vars[(vehicle, charger, hour)].varValue) for hour in range(charger.available_hours)]
                charging_schedule.append((vehicle, charger, schedule))

        total_cost_sum += value(total_cost)
        total_time_sum += value(total_time)

    avg_total_cost = total_cost_sum / num_samples
    avg_total_time = total_time_sum / num_samples

    return avg_total_cost, avg_total_time, charging_schedule

def print_charging_schedule(charging_schedule):
    print("Optimal Charging Schedule:")
    for vehicle, charger, schedule in charging_schedule:
        print("Vehicle:", vehicle.distance)
        print("Charger ID:", charger.charger_id)
        print("Charging Schedule:")
        for hour, charged in schedule:
            print("Hour:", hour, "| Charged:", charged * vehicle.charging_speed)
        print("----------------------")

# Define vehicles
vehicles = [
    Vehicle(distance=100, battery_capacity=200, current_charge=50, charging_speed=10, efficiency=20),
    Vehicle(distance=150, battery_capacity=300, current_charge=100, charging_speed=15, efficiency=20),
    Vehicle(distance=80, battery_capacity=150, current_charge=30, charging_speed=8, efficiency=20)
]

# Define chargers
chargers = [
    Charger(charger_id=1, available_hours=24),
    Charger(charger_id=2, available_hours=24),
    Charger(charger_id=3, available_hours=24)
]

# Define hourly prices
hourly_prices = [8, 10, 12, 15, 12, 10, 8, 10, 12, 15, 10, 8, 10, 12, 15, 10, 8, 12, 15, 12, 10, 8, 10, 12]

# Number of samples
num_samples = 100

# Solve the optimization problem
avg_total_cost, avg_total_time, charging_schedule = optimize_charging_schedule_stochastic(vehicles, chargers, hourly_prices, num_samples)

# Print results
print("Resultados de la Optimización Estocástica:")
print("Average Total Cost:", avg_total_cost)
print("Average Total Time:", avg_total_time)

# Print schedule
print_charging_schedule(charging_schedule)
